In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

In [ ]:
df = pd.read_pickle('../data/reddit_corpus_balanced_filtered.gzip', compression='gzip')

In [ ]:
link_amounts = []

for row in tqdm(df.iterrows(), desc='Evaluating Named Entities', total=len(df)):
    data = row[1]
    documents = data['documents']
    user_id = data['user_id']
    link_amount = [0, 0]
    
    for post in documents:
        for link in post[4]:
            # increase fn/rn count
            link_amount[link[1]] += 1
    
    link_amounts.append(link_amount)

df['link_amounts'] = link_amounts

In [ ]:
# removing users with less than 3 links
mask = [sum(i) >= 3 for i in df['link_amounts']]
print('Dropping', str(len(df)-sum(mask)), 'users')
df = df[mask]

In [ ]:
sum(df['fake_news_spreader'])/len(df)

In [ ]:
df['link_label'] = [int(2*(i[1])**(2)/sum(i) > 0.15) for i in df['link_amounts']]

sum(df['link_label'])/len(df)

In [ ]:
a, _c = np.unique(df['link_amounts'], return_counts=True)
fn_filt = [bool(int(2*i[1]**(2)/sum(i) > 0.15)) for i in a]
rn_filt = [not i for i in fn_filt]
plt.scatter([i[0]+1 for i in a[fn_filt]], [i[1]+1 for i in a[fn_filt]], marker='D',
            s=np.sqrt(_c[fn_filt])*10, c='orange', alpha=0.5, label='Misinformation Spreaders')
plt.scatter([i[0]+1 for i in a[rn_filt]], [i[1]+1 for i in a[rn_filt]], marker='D',
            s=np.sqrt(_c[rn_filt])*10, c='purple', alpha=0.5, label='Real News Spreaders')
plt.xscale('log')
plt.yscale('log')
plt.title('User labels based on link distribution')
plt.grid(True)
plt.ylabel('fake-news links (+1)')
plt.xlabel('real-news links (+1)')
plt.legend()
plt.savefig('re-label-full-fn.pdf', bbox_inches='tight');